# ROOT analyser

In [ ]:
%load_ext autoreload
%autoreload 2

import json
import cv2 as cv
import numpy as np

from src.viz.graphs import draw_map
from src.viz.images import imshow, scaled_imshow, draw_bbox
from src.utils.contours import warp_contour
from src.utils.images import calculate_color_coverage, crop_image, rotate_image
from src.utils.data import get_pdf_page, get_frame
from src.detection.elements import detect_from_reference, detect_dice_tray, detect_score_board, detect_clearings_and_buildings, detect_pawns
from src.detection.game import calculate_current_score, calculate_current_buildings_control, calculate_current_clearing_control

In [ ]:
large_imshow = lambda img_to_scale: scaled_imshow(img_to_scale, fx=0.8, fy=0.8)
simshow = lambda img_to_scale: scaled_imshow(img_to_scale, fx=0.3, fy=0.3)
mini_imshow = lambda img_to_scale: scaled_imshow(img_to_scale, fx=0.2, fy=0.2)

In [ ]:
mse = lambda a_1,a_2: np.mean((np.array(a_1)-np.array(a_2))**2)
_max_color_diff = mse((0,0,0),(255,255,255))
def color_sim(a_1,a_2):
    return 1 - (mse(a_1,a_2)/_max_color_diff)

In [ ]:
clip = lambda idx: f"clip_{idx}"
clip_mp4 = lambda idx: f"{clip(idx)}.mp4"

## Data

The input data was divided into 3 groups depending on the difficulty for example:
- easy: perfect top-down view, the game elements are not covered with hands when carrying them, the lighting is good
- medium: strong light at the side causing shadows,
- difficult: same as medium + a slightly angled camera, hands covering the pieces

There are 3 clips per difficulty. The data is located in a [Google Drive]((https://drive.google.com/drive/folders/1VrQ98TC5jPmWk1QYr3lUP3SGk_3_AEmx?usp=sharing)). We also resized the clips using the resize_data.py script, to speed up the detection process.


In [ ]:
DATA_DIR = "./data"
GAME_DATA_DIR = f"{DATA_DIR}/game_data"
RESULTS_DIR = "./results"
DIFFICULTIES = ["easy", "medium", "hard"]
CLIP_DIRS = dict([(diff, f"{DATA_DIR}/{diff}") for diff in DIFFICULTIES])
RESIZED_CLIP_DIRS = dict([(diff, f"{DATA_DIR}/{diff}/resized") for diff in DIFFICULTIES])
FIRST_FRAMES = dict([(diff,get_frame(f"{_dir}/{clip_mp4(0)}")) for diff, _dir in CLIP_DIRS.items()])
RESIZED_FIRST_FRAMES = dict([(diff,get_frame(f"{_dir}/{clip_mp4(0)}")) for diff, _dir in RESIZED_CLIP_DIRS.items()])
LOWER_ORANGE = np.array([0, 100, 100])
ORANGE = (48,91,198)
UPPER_ORANGE = np.array([20, 255, 255])
LOWER_DARK_BLUE = np.array([100, 50, 50])
BLUE = (92,38,15)
UPPER_DARK_BLUE = np.array([140, 255, 255])

In [ ]:
mini_imshow(FIRST_FRAMES["easy"])

In [ ]:
mini_imshow(FIRST_FRAMES["medium"])

In [ ]:
mini_imshow(FIRST_FRAMES["hard"])

In [ ]:
simshow(np.concatenate([RESIZED_FIRST_FRAMES[diff] for diff in DIFFICULTIES], axis=1))

The game is played between 2 factions: Eyrie Dynasties (blue birds), Marquise de Cat (orange cats). The board is a Winter Map. Because the clearings in the forest are barely differentiable, a mask was created to help with detecting static elements of the board. 

In [ ]:
board_mask = cv.imread(f"{GAME_DATA_DIR}/board_mask.png")

simshow(board_mask)

- The red indicates the where the score track is. 
- The green defines where craftable items are.
- The blue shows where the clearing approximately are, with the black squares showing where building spaces are.

JSON was created to define paths on the map, done purely for drawing a graph of the map.

In [ ]:
with open(f"{GAME_DATA_DIR}/board_info.json", "r") as info_file:
    board_info = json.load(info_file)

draw_map(board_info)

To help with detection a print and play set is used with all the elements taken from [PnP PARADISE](https://www.pnpparadise.com/set1/root).

In [ ]:
board_ref = get_pdf_page(f"{GAME_DATA_DIR}/board.pdf")

simshow(board_ref)

## Milestone 1

In this phase, the following things were detected:
- the black dice tray along with the dice on it
- the board

### Dice tray detection

In [ ]:
image = FIRST_FRAMES["easy"]
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

In [ ]:
mini_imshow(gray)

The dice tray is all black so a simple threshold was performed.

In [ ]:
tray_cont, dice1_cont, dice2_cont, img_cont = detect_dice_tray(image, 50, draw_contours=True)
imshow(img_cont)

In [ ]:
tray = crop_image(image, tray_cont)

dice1 = crop_image(image, dice1_cont)
dice2 = crop_image(image, dice2_cont)

mini_imshow(tray)
imshow(dice1)
imshow(dice2)

### Board detection

Detecting the board was harder as it has much more details.

In [ ]:
board_gray = cv.GaussianBlur(cv.cvtColor(board_ref, cv.COLOR_BGR2GRAY),(7,7),0)

simshow(board_gray)

To achieve this steps descriptors are used, in particular the SIFT detector. To quickly match the descriptors FLANN algorithm is used.

In [ ]:
M_board, board_cont, img_matches = detect_from_reference(image, board_ref, draw_matches=True)
simshow(img_matches)

In [ ]:
crop_board = crop_image(image, board_cont)
crop_board = rotate_image(crop_board,0.8)
simshow(crop_board)

This code doesn't have to be run much, because the board should not a lot move in the clips

In the milestone 1, there were also attempts to segment the image using a Gaussian Mixture, but they were quite slow and not effective

## Further Progress

### Tracking Game Score

Game score is tracked in the lower half of the board, by blue and orange counters. They are found using the red part of the mask.

In [ ]:
simshow(np.concatenate([board_ref, board_mask], axis=1))

In [ ]:
cell_contours, score_cont = detect_score_board(board_ref, board_mask[:,:,2])
score_x,score_y,_,_ = cv.boundingRect(score_cont)
test_score_crop = crop_image(board_ref,score_cont)
imshow(test_score_crop)

In [ ]:
cell_contours = list(map(lambda cont: warp_contour(cont, M_board), [cont + [score_x,score_y] for cont in cell_contours]))
simshow(cv.drawContours(np.copy(image), cell_contours, -1,(255,0,0),2))

In [ ]:
imshow(crop_image(image, cell_contours[0]))

In [ ]:
for i, cell in enumerate(cell_contours):
    print(f"Cell {i}: ", end=" ")
    print(f"Orange: {calculate_color_coverage(crop_image(image, cell),LOWER_ORANGE,UPPER_ORANGE):.2%}", end=" ")
    print(f"Blue: {calculate_color_coverage(crop_image(image, cell),LOWER_DARK_BLUE,UPPER_DARK_BLUE):.2%}")

In [ ]:
score = calculate_current_score(image, cell_contours, (LOWER_ORANGE,UPPER_ORANGE), (LOWER_DARK_BLUE,UPPER_DARK_BLUE))
score

### Tracking Card Pile

Similarly to the board the cards are found using descriptors

In [ ]:
card_ref = get_pdf_page(f"{GAME_DATA_DIR}/card_reverse.pdf")
imshow(card_ref)

In [ ]:
M_card, card_cont, img_matches = detect_from_reference(image, card_ref, distance=0.5, draw_matches=True)

In [ ]:
mini_imshow(img_matches)

In [ ]:
crop_card = crop_image(image, card_cont)
imshow(crop_card)

### Tracking Buildings

Buildings are detected using the holes in the clearing mask, as they are stationary.

In [ ]:
_, building_contours = detect_clearings_and_buildings(board_mask[:,:,0])
building_contours = [building for clearing in building_contours.values() for building in clearing]
building_contours = list(map(lambda cont: warp_contour(cont, M_board), building_contours))
simshow(cv.drawContours(np.copy(image), building_contours, -1,(255,0,0),2))

When percentage of particular color in the building is higher than 33% it is considered to be present.

In [ ]:
for i, building in enumerate(building_contours):
    b_crop = crop_image(image, building)
    print(f"Building {i}: ", end=" ")
    print(f"Orange: {calculate_color_coverage(b_crop,LOWER_ORANGE,UPPER_ORANGE):.2%}", end=" ")
    print(f"Blue: {calculate_color_coverage(b_crop,LOWER_DARK_BLUE,UPPER_DARK_BLUE):.2%}")
    imshow(b_crop)

In [ ]:
ob, bb = calculate_current_buildings_control(image, building_contours, (LOWER_ORANGE,UPPER_ORANGE), (LOWER_DARK_BLUE,UPPER_DARK_BLUE))
sum(ob), sum(bb)

### Clearings and building order standardization

The clearing contours will be found based on the clearings mask in order for their order to be stable. Then the contours will be properly warped into the board

In [ ]:
clearings,buildings = detect_clearings_and_buildings(board_mask[:,:,0])

In [ ]:
c = np.repeat(np.copy(board_mask[:,:,0])[:,:,np.newaxis],3,axis=2)
for i,cont in enumerate(clearings):
    x,y,w,h = cv.boundingRect(cont)
    c = cv.putText(c,str(i),(x,y),cv.FONT_HERSHEY_COMPLEX,2,(0,255,0),2)
cv.drawContours(c,clearings,-1,(0,255,0),5)
cv.drawContours(c,buildings[8],-1,(255,0,0),5)
simshow(c)

### Pawn Tracking

To achieve pawn tracking we will take a mask of the clearings


In [ ]:
warped_clearing_mask = cv.warpPerspective(board_mask[:,:,0],M_board,(image.shape[1], image.shape[0]))
warped_clearings = [warp_contour(cont,M_board) for cont in clearings]

In [ ]:
pawn_crop_test = crop_image(cv.bitwise_and(image,image,mask=warped_clearing_mask),board_cont)
mini_imshow(pawn_crop_test)

And convert the color range to HSV

In [ ]:
mini_imshow(cv.cvtColor(pawn_crop_test,cv.COLOR_BGR2HSV))

And the elements in orange range and blue range are cropped. Because other elements get in the way, the pawn counting is divided into 2 parts.

1. We divide the checked area by the biggest area and see if it is smaller than some sensitivity threshold.
2. Per clearing we take the given area and divide it by areas 'derivative' and check for another threshold.

In [ ]:
op, bp = detect_pawns(image, warped_clearing_mask, warped_clearings, (LOWER_ORANGE,UPPER_ORANGE), (LOWER_DARK_BLUE,UPPER_DARK_BLUE))

for i in range(12):
    print(f"Clearing {i}: ", end=" ")
    print(f"Orange: {len(op[i])}", end=" ")
    print(f"Blue: {len(bp[i])}")
    imshow(crop_image(image, warped_clearings[i]))

In [ ]:
oc, bc = calculate_current_clearing_control(op, bp)
sum(oc), sum(bc)

In [ ]:
rects = [cv.boundingRect(clearing) for clearing in warped_clearings]
orange_pawns = [cv.boundingRect(pawn + [rects[c_idx][0], rects[c_idx][1]]) for c_idx, clearing in op.items() for pawn in clearing]
blue_pawns = [cv.boundingRect(pawn + [rects[c_idx][0], rects[c_idx][1]]) for c_idx, clearing in bp.items() for pawn in clearing]

img_cont = np.copy(image)
cv.drawContours(img_cont, warped_clearings, -1, (0,255,0), 2)

for pawns, color in ((orange_pawns, ORANGE), (blue_pawns, BLUE)):
    for rect in pawns:
        draw_bbox(img_cont, rect, color)

imshow(img_cont)

This solution is not perfect but good enough.

The pawns tracking helps us complete tracking of the game state by determining the control of each clearing and the number of pawns of each of them.

## Putting it all together

### Tracking algorithms and design decisions

When it comes to tracking, we found that on the higher resolutions the tracking algorithm using CSRT performed much better. This also applies to re-detection of objects.

Frequently moving elements are tracked using CSRT algorithm e.g. Dice and the Cards. To counter the tracking algorithm occasionally losing them, they are re-detected more frequently compared to stationary objects.

Things that change in stationary places, tracked by much simpler algorithms, with the stationary objects they are in being re-detected much less frequently to counter occasional camera movement.

### Tracking game state and events

The game state is tracked by detecting the following events:

1. Game score
2. Number of buildings in each clearing
3. Number of pawns in each clearing
4. Control of each clearing

Which describe the most important part of the game state, especially for these 2 factions

There are several events that we detect:
1. Dice rolling
2. Buildings change
3. Score change
4. Drawing of a card by some player
5. Pawns change 

For some of them the change in average state over time is measured to decrease their sensitivity. 

In [ ]:
img_with_state = np.copy(image)

# Dice tray and dice
cv.drawContours(img_with_state, [tray_cont], -1, (0,122,0), 2)
dice_1, dice_2 = cv.boundingRect(dice1_cont), cv.boundingRect(dice2_cont)
draw_bbox(img_with_state, dice_1, (0,0, 255))
cv.putText(img_with_state, "Dice 1", (dice_1[0] + dice_1[2]//2 - 10, dice_1[1] - 10), cv.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
draw_bbox(img_with_state, dice_2, (0,0, 255))
cv.putText(img_with_state, "Dice 2", (dice_2[0] + dice_2[2]//2 - 10, dice_2[1] - 10), cv.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)

# Board
cv.drawContours(img_with_state, [board_cont], -1, (0,122,0), 2)

# Score Board
cv.drawContours(img_with_state, cell_contours, -1, (0,122,0), 2)
cv.drawContours(img_with_state, [cell_contours[score[1]]], -1, BLUE, 3)
cv.drawContours(img_with_state, [cell_contours[score[0]]], -1, ORANGE, 3)

# Cards
cards = cv.boundingRect(card_cont)
cv.drawContours(img_with_state, [card_cont], -1, (0,122,0), 2)
draw_bbox(img_with_state, cards, (0,0, 255))
cv.putText(img_with_state, "Cards", (cards[0] + cards[2]//2 - 10, cards[1] - 10), cv.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)

# Buildings
cv.drawContours(img_with_state, building_contours, -1, (0,122,0), 2)
cv.drawContours(img_with_state, [building_contours[i] for i in range(len(building_contours)) if bb[i]], -1, BLUE, 3)
cv.drawContours(img_with_state, [building_contours[i] for i in range(len(building_contours)) if ob[i]], -1, ORANGE, 3)

# Clearings and pawns
cv.drawContours(img_with_state, warped_clearings, -1, (0,122,0), 2)
cv.drawContours(img_with_state, [warped_clearings[i] for i in range(len(warped_clearings)) if bc[i]], -1, BLUE, 3)
cv.drawContours(img_with_state, [warped_clearings[i] for i in range(len(warped_clearings)) if oc[i]], -1, ORANGE, 3)

for idx, clearing in enumerate([cv.boundingRect(warped_clearings[i]) for i in range(len(warped_clearings))]):
    cv.putText(img_with_state, str(len(op[idx])), (clearing[0] + clearing[2]//2 - 30, clearing[1] - 10), cv.FONT_HERSHEY_COMPLEX, 1, ORANGE, 3)
    cv.putText(img_with_state, ":", (clearing[0] + clearing[2]//2 - 7, clearing[1] - 10), cv.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 3)
    cv.putText(img_with_state, str(len(bp[idx])), (clearing[0] + clearing[2]//2 + 10, clearing[1] - 10), cv.FONT_HERSHEY_COMPLEX, 1, BLUE, 3)

for pawns, color in ((orange_pawns, ORANGE), (blue_pawns, BLUE)):
    for rect in pawns:
        draw_bbox(img_with_state, rect, color, 3)

# Displaying events
for idx, event in enumerate((
        "Dice 1 rolled", 
        "Dice 2 rolled", 
        "Card drawn - Orange",
        f"Score - Orange {score[0]} Blue {score[1]}", 
        f"Buildings Constructed - Orange {sum(ob)} Blue {sum(bb)}", 
        f"Pawns Placed - Orange {len(orange_pawns)} Blue {len(blue_pawns)}")
):
    cv.putText(img_with_state, event, (10, 50 + idx * 75), cv.FONT_HERSHEY_COMPLEX, 2, (0, 0, 0), 12)
    cv.putText(img_with_state, event, (10, 50 + idx * 75), cv.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 3)

        
imshow(img_with_state)

## Results

### Easy

For easy clips tracking is overall good, but we've noticed a few bugs:
1. Sometimes, when pawn is placed slightly in the building slot, the system will cout it as building
2. Some pawns are not counted as ones
3. Restarting tracking of the mobile components often times breaks after movement, specially for dice

### Medium

For medium clips tracking is worse than on easy, and we've noticed:

1. Tracking of pawns and buildings gets really unstable, especially when there are many placed
2. Detecting pawns and building gets problematic because of the light and shadows
3. Detecting dice and the dice tray is bugging out, sometimes detecting the blue player's game card

### Hard

For hard clips tracking works suprisingly well, given that it is rotated, with hard light conditions. We've noticed following bugs:

1. Detecting dice tray and dice gets almost impossible, oftentimes bugging out
2. There are problems with detecting pawns and buildings, game has problems with tracking its clearing control state